In [ ]:
#packages inlezen
from netCDF4 import Dataset, chartostring
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import os, sys, time
import hkvsobekpy as hkv
import numpy as np
import plotly.io as pio
import plotly.express as px
import plotly.subplots as sp
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import datetime
from tqdm import tqdm
%matplotlib inline

In [ ]:
#paden opgeven naar modellen
#alleen de modelmapnaam aanpassen

#DHydro_Model_his= os.path.abspath('../03_Model/06_1D2DmodelGoorloop.dsproj_data/FlowFM/output/FlowFM_his.nc') #Goorloop
#DHydro_Model_map = os.path.abspath('../03_Model/06_1D2DmodelGoorloop.dsproj_data/FlowFM/output/FlowFM_map.nc') #Goorloop
DHydro_Model_his= os.path.abspath('../03_Model/Hwet2Dtestrun2.dsproj_data/FlowFM/output/FlowFM_his.nc') #Hertogswetering
DHydro_Model_map = os.path.abspath('../03_Model/Hwet2Dtestrun2.dsproj_data/FlowFM/output/FlowFM_map.nc') #Hertogswetering

opslaan = True

In [ ]:
#his en map bestand uit D-Hydro uitlezen
his = Dataset(DHydro_Model_his)
network = Dataset(DHydro_Model_map)

## Data uitlezen uit D-Hydro voor waterstanden op observatiepunten

In [ ]:
#D-Hydro 1D
node_id = chartostring(network['mesh1d_node_id'][:])
node_id = [nodeid.rstrip() for nodeid in node_id]

In [ ]:

ref_time = ' '.join(his['time'].units.split()[2:4])
print('reference time D-Hydro:',ref_time)
tijd = his['time'][:]

ref = datetime.datetime.strptime(ref_time, "%Y-%m-%d %H:%M:%S")

index_his = [ref + datetime.timedelta(seconds=each) for each in tijd]

locaties = chartostring(his['station_id'][:])
locaties = [str(locatie).rstrip() for locatie in locaties]

#waterstanden measurement stations
data = his['waterlevel'][:]
D_meas_WL = pd.DataFrame(data= data, index=index_his, columns = locaties )

In [ ]:
#typ hier plotly om interactieve grafieken te genereren, anders worden er statische afbeeldingen genereert via matplotlib (makkelijker te exporteren)
plot = 'plotly' 

#waterstanden measurement stations
for locatie in locaties:
    if plot == 'plotly':
        fig = px.line(D_meas_WL, x = D_meas_WL.index,y = [D_meas_WL.loc[:,locatie]], title='Waterstand bij '+locatie,
                      labels={"value": "Waterstand [m NAP]",  "index": "Datum"},
                     template='plotly') #template aannpassen, opties zijn:     
        #['ggplot2', 'seaborn', 'simple_white', 'plotly',
        #'plotly_white', 'plotly_dark', 'presentation', 'xgridoff',
        #'ygridoff', 'gridon', 'none']

        #verschillende opties voor aan en uitzetten van hoverinfo en opmaak ervan
        fig.update_traces(mode="lines", hovertemplate=None)
        fig.update_layout(hovermode="x unified")
        
        if opslaan ==True:
            pio.write_image(fig,'../02_Validatie/Hertogswetering/Meetpunten/svg/meetpunt_'+ locatie +'.svg', engine='orca')
            fig.write_html('../02_Validatie/Hertogswetering/Meetpunten/html/meetpunt_'+ locatie +'.html', include_plotlyjs="cdn", full_html=False)
        
        fig.show()
    else:
        fig,ax = plt.subplots(figsize=(20, 10))
        ax.plot(D_meas_WL.index,D_meas_WL.loc[:,locatie])
        ax.grid()
        ax.tick_params(labelsize=16)
        ax.legend(['D-Hydro'], fontsize = 16)
        ax.set_xlabel('Datum',fontsize=16)
        ax.set_ylabel('waterstand [m NAP]',fontsize=16)
        ax.set_title(locatie,fontsize=16)